In [37]:
from matplotlib import dates as mdate
from matplotlib import pyplot as plt
import matplotlib.animation as animation


import pandas as pd

from time import sleep

from IPython.display import clear_output

import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import VarianceThreshold

import random

import joblib

In [38]:
aOptimal = pd.read_csv("../../Trainable data/Anthony/Anthony_Optimal_Training_Data.csv")
aInoptimal = pd.read_csv("../../Trainable data/Anthony/Anthony_Inoptimal_Training_Data.csv")

row0 = aOptimal.iloc[0].values.tolist()[1:]

xarr = []
yarr = []

In [39]:
# aOptimal

In [40]:
u = 1
while (u < aOptimal.shape[0]):
    
    row0 = aOptimal.iloc[u].values.tolist()[1:]
    
    rfin = []

    p = 0
    while (p < len(row0)):

        row0[p] = row0[p][1:-1]
        row0[p] = row0[p].split(",")

        k = 0
        while (k < len(row0[p]) and k < 155):
            # effectively flatten it
#             row0[p][k] = float(row0[p][k])
            rfin.append(float(row0[p][k]))
            k += 1
        
        # print the length of features in that stroke
        # print(k)

        p += 1
    
    u += 1
    
    xarr.append(rfin)
    yarr.append(0)
    

In [41]:
u = 0
while (u < aInoptimal.shape[0]):
    
    row0 = aInoptimal.iloc[u].values.tolist()[1:]
    
    rfin = []

    p = 0
    while (p < len(row0)):

        row0[p] = row0[p][1:-1]
        row0[p] = row0[p].split(",")

        k = 0
        while (k < len(row0[p]) and k < 155):
            # effectively flatten it
#             row0[p][k] = float(row0[p][k])
            rfin.append(float(row0[p][k]))
            k += 1
        
        # print the length of features in that stroke
        # print(k)

        p += 1
    
    u += 1
    
    xarr.append(rfin)
    yarr.append(1)

In [42]:
X = np.array(xarr)
y = np.array(yarr)

# X

In [43]:
X_train, X_test, y_train, y_test = train_test_split(

    X, y, test_size=0.33, random_state=random.randint(0, 50))

In [44]:
ET_model = make_pipeline(StandardScaler(), ExtraTreesClassifier()) 

ET_model.fit(X_train, y_train)

predict_y = ET_model.predict(X_test)

In [45]:
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0

i = 0
while (i < len(predict_y)):
    pred = predict_y[i]
    real = y_test[i]
    
    if (pred == real and pred == 1):
        true_pos += 1
    elif (pred == real and pred == 0):
        true_neg += 1
    elif (pred != real and real == 0):
        false_pos += 1
    else:
        false_neg += 1
    
    i += 1
    
    
precision = (true_pos)/(true_pos+false_pos)
recall = (true_pos)/(true_pos+false_neg)
accuracy = (true_pos+true_neg)/(true_pos+true_neg+false_pos+false_neg)
fscore = 2*(precision*recall)/(precision+recall)

In [46]:
precision, recall, accuracy, fscore

(1.0, 1.0, 1.0, 1.0)

In [47]:
print("Predictions:", predict_y)
print("True labels:", y_test)

Predictions: [0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 1]
True labels: [0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 0 1]


# Feature Analysis

## Mutual Information Scores (measure dependecy between features and target variable)

In [48]:
X = pd.DataFrame(X, columns=[f"Feature_{i}" for i in range(X.shape[1])])

# create dataframe of each feature and its importance
mutual_info = mutual_info_classif(X, y)
feature_importance = pd.DataFrame({
    'Feature' : X.columns,
    'Importance' : mutual_info
}).sort_values(by='Importance', ascending=False)

print(feature_importance)

           Feature  Importance
2522  Feature_2522    0.675993
2335  Feature_2335    0.675993
2336  Feature_2336    0.675993
2337  Feature_2337    0.675993
2331  Feature_2331    0.675993
...            ...         ...
1805  Feature_1805    0.000000
1658  Feature_1658    0.000000
1800  Feature_1800    0.000000
1796  Feature_1796    0.000000
1766  Feature_1766    0.000000

[2790 rows x 2 columns]


In [49]:
# drop low-importance features (importance < 0.01)
low_importance_features = feature_importance[feature_importance['Importance'] < 0.01]['Feature']
X_reduced = X.drop(columns=low_importance_features)

## Variance Threshold (Can remove features with low variance as they usually add little value)

In [50]:
selector = VarianceThreshold(threshold=0.01) # threshold can be adjusted to remove more
X_reduced = selector.fit_transform(X)
print("Reduced Features Shape:", X_reduced.shape)

Reduced Features Shape: (52, 2697)


## ExtraTreeClassifier Feature Importance Direct Analysis

In [51]:
# access just the ExtraTreesClassifier from the pipeline
extra_trees = ET_model.named_steps['extratreesclassifier']  

# create dataframe of each feature and its importance
ETC_feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': extra_trees.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(ETC_feature_importance)



           Feature  Importance
2786  Feature_2786    0.021000
2350  Feature_2350    0.016222
906    Feature_906    0.013128
2767  Feature_2767    0.011787
892    Feature_892    0.011787
...            ...         ...
2760  Feature_2760    0.000000
0        Feature_0    0.000000
2757  Feature_2757    0.000000
2756  Feature_2756    0.000000
4        Feature_4    0.000000

[2790 rows x 2 columns]


In [52]:
# ETC_low_importance_features = ETC_feature_importance[ETC_feature_importance['Importance'] < 0.01]['Feature']
# ETC_X_reduced = X.drop(columns=ETC_low_importance_features)